# Plotting Circular Dichroism Data

## Prerequisites

In [ ]:
import pandas as pd

In [ ]:
# Necessary libraries
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

In [ ]:
# User authentication
auth.authenticate_user()

# Credentials
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Google Sheets file opening
sheet_name = 'file_name'
sheet = gc.open(sheet_name)

# Pages opening inside the file
page_name1 = 'page_name_01'
page1 = sheet.worksheet(page_name1)
page_name2 = 'page_name_02'
page2 = sheet.worksheet(page_name2)

In [ ]:
# Obtaining pages values
data1 = page1.get_all_values()
data2 = page2.get_all_values()

In [ ]:
# Pandas dataframe creation
df1 = pd.DataFrame(data1[1:], columns=data1[0])
df2 = pd.DataFrame(data2[1:], columns=data2[0])

In [ ]:
# Replace commas with dots in all columns (ONLY IF NECESSARY)
df1 = df1.replace(',', '.', regex=True)
df2 = df2.replace(',', '.', regex=True)

## Baseline subtraction

In [ ]:
# Convert all columns to numeric
df1 = df1.apply(pd.to_numeric, errors='coerce')
df2 = df2.apply(pd.to_numeric, errors='coerce')

In [ ]:
# Subtract blank values from DF1
df1['pep_buffer_pH40_w/o_blank'] = df1["pep_buffer_pH40"] - df1['buffer_40']
df1['pep_buffer_pH50_w/o_blank'] = df1["pep_buffer_pH50"] - df1['buffer_50']

In [ ]:
# Subtract blank values from DF2
df2['pep_buffer_pH40_w/o_blank'] = df2["pep_buffer_pH40"] - df2['buffer_40']
df2['pep_buffer_pH50_w/o_blank'] = df2["pep_buffer_pH50"] - df2['buffer_50']

## Plotting

In [ ]:
# Importing libraries
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib as mpl

In [ ]:
#Setting style
plt.style.use('seaborn-ticks')

# Savitzky-Golay filter
from scipy.signal import savgol_filter

#Setting subplots
fig = plt.figure(figsize=(8,11), dpi=300)
ax1 = plt.subplot2grid((1,2), (0,0))
ax2 = plt.subplot2grid((1,2), (0,1))

#############################################################################################
#ax1 definitions
#############################################################################################

for axis in ['top', 'bottom', 'left', 'right']:
  ax1.spines[axis].set_linewidth(1) #border size

ax1.axhline(linewidth=1, color='black')

#smoothing aplication - Savitzky-Golay filter
yhat1 = savgol_filter(df1['pep_buffer_pH40_w/o_blank'], 51, 4)
yhat2 = savgol_filter(df1['pep_buffer_pH50_w/o_blank'], 51, 4)

#axes X and Y data
cmap=plt.cm.viridis #viridis colormap is color-blind friendly
l1, =ax1.plot(df1['Wavelength'], yhat1, lw=1, color=cmap(0.1))
l2, =ax1.plot(df1['Wavelength'], yhat2, lw=1, color=cmap(0.3))

#axes parameters
ax1.set_title('Title 1', fontsize = 14, weight='normal', pad=20) #title
ax1.set_ylabel('Elipticity x 10$^5$ (degrees.cm$^2$/dmol)', fontsize=8, weight='bold') #y label
ax1.set_xlabel('Wavelength (nm)', fontsize=8, weight='bold') #x label
ax1.set_xlim(190, 260) #x limits

ax1.legend([l1, l2],
           ['caption 1', 'caption 2'],
           loc='upper right', #captions location
           frameon=False,
           ncol=1,
           shadow=False,
           borderpad=1,
           fontsize=8)

#############################################################################################
#ax2 definitions
#############################################################################################

for axis in ['top', 'bottom', 'left', 'right']:
  ax2.spines[axis].set_linewidth(1)

ax2.axhline(linewidth=1, color='black')

#smoothing aplication - Savitzky-Golay filter
yhat3 = savgol_filter(df2['sds_pep_pH40_w/o_blank'], 51, 4)
yhat4 = savgol_filter(df2['sds_pep_pH50_w/o_blank'], 51, 4)

#axes X and Y data - in this example, three curves in each curve
ll1, =ax2.plot(df2['Wavelength'], yhat3, lw=1, color=cmap(0.1))
ll2, =ax2.plot(df2['Wavelength'], yhat4, lw=1, color=cmap(0.3))

#axes parameters
ax2.set_title('Title 2', fontsize = 14, weight='normal', pad=20)
ax2.set_ylabel('Elipticity x 10$^5$ (degrees.cm$^2$/dmol)', fontsize=8, weight='bold')
ax2.set_xlabel('Wavelength (nm)', fontsize=8, weight='bold')
ax2.set_xlim(190,260)

ax2.legend([ll1, ll2],
           ['caption 1', 'caption 2'],
           frameon=False,
           ncol=1,
           shadow=False,
           borderpad=1,
           fontsize=8)

#############################################################################################
#Final Figure
#############################################################################################

#the coordinates and font size are encouraged to be exchanged. If necessary, add new topics;
ax1.text(-0.3, 1.2, ' a)', transform=ax1.transAxes, fontsize=18, va='top')
ax2.text(-0.3, 1.2, ' b)', transform=ax2.transAxes, fontsize=18, va='top')

#saving the figures
plt.tight_layout()